In [1]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import minimize_scalar

from SALib.sample import saltelli,latin
import pandas as pd
import numpy as np

from scipy.interpolate import griddata


# Input Variables
# k (capital) 1-10 uniform, 1-15 uniform for random sample 
# θ (perceived shock factor) 0.1-1 uniform
# σ (risk averseness) normal centered around 1.08
# α (aptitude/human capital) normal centered around 1



In [32]:
# Generate Samples

# Saltelli Sample

n=1024
N=5000
seed=1
#2d+2=10

problem={   "names": ["Alpha","k","Sigma","Theta"], 
            "num_vars":4,
            "bounds":[[1.08,0.074],[0.1,10],[1,0.5],[0.1,1]],
            "dists":["norm","unif","norm","unif"]}
            #"seed":seed

S_sample=saltelli.sample(problem,n)


S_sampledf=pd.DataFrame(S_sample, columns=["Alpha","k","Sigma","Theta"])

S_sampledf["Sigma"]=np.round(S_sampledf["Sigma"],1)
S_sampledf["Theta"]=np.round(S_sampledf["Theta"],1)
S_sampledf.index.name="AgentID"



# # Latin Hypercube Sample
# LH_sample=latin.sample(problem,N,seed)

# LH_sampledf=pd.DataFrame(LH_sample, columns=["Alpha","k","Sigma","Theta"])

# LH_sampledf["Sigma"]=np.round(LH_sampledf["Sigma"],1)
# LH_sampledf["Theta"]=np.round(LH_sampledf["Theta"],1)
# LH_sampledf.index.name="AgentID"

# # Random Sample
# np.random.seed(seed)
# alphas = np.random.normal(loc = 1.08, scale = 0.074, size = N) #list of agent alphas, effectively ability/human capital
# capital = np.random.uniform(low = 0.1, high = 10, size = N) #list of agent initial capital amounts
# sigmas=np.round(np.random.normal(loc = 1, scale = 0.5,size=N),1) #list of agent sigmas, effectively risk-averseness
# thetas= np.round(np.random.uniform(low=0.1, high=1, size=N),1) #percieved theta (cannot be zero or the optimizer breaks because k<) maybe one day this can be spatial?
# R_sampledf=pd.DataFrame({"Alpha":alphas,"k":capital,"Sigma":sigmas,"Theta":thetas})

/var/folders/73/xt36gx2n50nfg3478nvjbx100000gn/T/ipykernel_92434/191297959.py:16: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  S_sample=saltelli.sample(problem,n)


In [33]:
# global variables for Bellman equation
𝛿 = 0.08 #depreciation
β = 0.95 #discount factor


TechTable = {#contains values for 0:gamma 1:cost 2:theta
# VMG things get stuck in a while loop if the gamma is less than 0.3 
# (tried 0.2) not sure yet if/how this will be problematic 
# Also important to make sure values are in the correct order
# i.e. that the threshold between medium and high is at a 
# higher k than the threshold between low and medium 
# This can be checked with k_threshold.py

    "low":   [0.3,  0   ],
    "medium":[0.35, 0.15],
    "high":  [0.45, 0.65]}

TechTableArray = np.array([[ 0.3,  0 ],[0.35, 0.15],[0.45, 0.65]])

AdapTable = {
    # contains values for 0:theta 1:cost 
    # (for consideration:effort? type? design life?)
    "none":   [  0, 0   ],
    "good":   [0.5, 0.25],
    "better": [0.9, 0.45]}

AdapTableArray = np.array([[ 0,  0 ],[0.5, 0.25],[0.9, 0.45]])



# Define Optimization Routine:


def maximize(g, a, b, args):
    """
    From: https://python.quantecon.org/optgrowth.html (similar example 
    https://macroeconomics.github.io/Dynamic%20Programming.html#.ZC13-exBy3I)
    Maximize the function g over the interval [a, b].

    The maximizer of g on any interval is
    also the minimizer of -g.  The tuple args collects any extra
    arguments to g.

    Returns the maximum value and the maximizer.
    """

    objective = lambda x: -g(x, *args)
    result = minimize_scalar(objective, bounds=(a, b), method='bounded')
    maximizer, maximum = result.x, -result.fun
    return maximizer, maximum

def utility(c, σ, type="isoelastic"):
    if type == "isoelastic":
        if σ ==1:
            return np.log(c)
        else:
            return (c**(1-σ)-1)/(1-σ)

    else:
        print("Unspecified utility function!!!")


def income_function(k,α): 
    f = []
    for i in TechTable.keys(): 
        #in the end, they may need their own tech tables
        entry = α * k**TechTable[i][0] - TechTable[i][1]
        f.append(entry)
    return max(f)

def adaptation_function(θ,i_a):
    
    for i in AdapTable.keys(): 
        #in the end, they should have their own adaptation tables
        if AdapTable[i][1] <= i_a:
            m = AdapTable[i][0]
        else:
            break
    return θ + m * (1-θ)



class BellmanEquation:
     #Adapted from: https://python.quantecon.org/optgrowth.html
    def __init__(self,
                 u,            # utility function
                 f,            # production function
                 k,            # current state k_t
                 θ,            # given shock factor θ
                 σ,            # risk averseness
                 α,            # human capital
                 i_a,          # adaptation investment
                 m,            # protection multiplier
                 β=β,          # discount factor
                 𝛿=𝛿,          # depreciation factor 
                 name="BellmanNarrowExtended"):

        self.u, self.f, self.k, self.β, self.θ, self.𝛿, self.σ, self.α, self.i_a, self.m, self.name = u, f, k, β, θ, 𝛿, σ, α, i_a, m, name

        # Set up grid
        
        startgrid=np.array([1.0e-7,1,2,3,4,5,6,7,8,9,10,k+100])

        ind=np.searchsorted(startgrid, k)
        self.grid=np.concatenate((startgrid[:ind],np.array([k*0.99999, k]),
                                 startgrid[ind:]))

        self.grid=self.grid[self.grid>i_a]

        # Identify target state k
        self.index = np.searchsorted(self.grid, k)-1
    
    def value(self, c, y, v_array):
        """
        Right hand side of the Bellman equation.
        """

        u, f, β, θ, 𝛿, σ, α, i_a, m = self.u, self.f, self.β, self.θ, self.𝛿, self.σ, self.α, self.i_a, self.m

        v = interp1d(self.grid, v_array, bounds_error=False, 
                     fill_value="extrapolate")
        
        return u(c,σ) + β * v((θ + m * (1-θ)) * (f(y,α) - c - i_a + (1 - 𝛿) * y))



def update_bellman(v, bell):
    """
    From: https://python.quantecon.org/optgrowth.html (similar example
    https://macroeconomics.github.io/Dynamic%20Programming.html#.ZC13-exBy3I)
    
    The Bellman operator.  Updates the guess of the value function
    and also computes a v-greedy policy.

      * bell is an instance of Bellman equation
      * v is an array representing a guess of the value function

    """
    v_new = np.empty_like(v)
    v_greedy = np.empty_like(v)
    
    for i in range(len(bell.grid)):
        y = bell.grid[i]
        # Maximize RHS of Bellman equation at state y
        
        c_star, v_max = maximize(bell.value, min([1e-8,y*0.00001]), 
                                 y-bell.i_a, (y, v))
        #VMG HELP! can anyone check that (1) subtracting i_a and 
        # (2) omitting any grid values less than i_a 
        # will not be problematic? The only thing I can come up with
        # is if i_a is greater than k*0.99999
        # which_bellman() now accounts for that case. Whole thing 
        # could use refinement.
      
        v_new[i] = v_max
        v_greedy[i] = c_star

    return v_greedy, v_new

def which_bellman(agentinfo):
    """
    Solves bellman for each affordable adaptation option.
    """
    feasible=[]


    for option in agentinfo.adapt:
        if option[1]>=(income_function(agentinfo.k,agentinfo.α)+(1-𝛿)*agentinfo.k)*0.99998:
            # ensures that the gridpoint
            # just below income, income*0.99999, is included
            pass
        else:
            #print(f'working theta = {agentinfo.θ + option[0] *\
            #  (1-agentinfo.θ)}, i_a= {option[1]}, k= {agentinfo.k}')
            c,v=solve_bellman(BellmanEquation(u=utility, 
                              f=income_function, k=agentinfo.k, 
                              θ=agentinfo.θ, σ=agentinfo.σ, 
                              α=agentinfo.α, i_a=option[1],m=option[0]))
            feasible.append([v,c,option[1],option[0]])

    best=min(feasible)

    
    return feasible



def solve_bellman(bell,
                  tol=1,
                  min_iter=10,
                  max_iter=1000,
                  verbose=False):
    """
    From: https://python.quantecon.org/optgrowth.html (similar example
    https://macroeconomics.github.io/Dynamic%20Programming.html#.ZC13-exBy3I)
    
    Solve model by iterating with the Bellman operator.

    """


    # Set up loop

    v = bell.u(bell.grid,bell.σ)  # Initial condition
    i = 0
    error = tol + 1
    max_error = tol*10 + 1

    while (i < max_iter and (error > tol or max_error > tol*10)) or (i < min_iter):
        v_greedy, v_new = update_bellman(v, bell)
        error = np.abs(v[bell.index] - v_new)[bell.index]
        max_error = np.max(np.abs(v - v_new))
        i += 1
        # if verbose and i % print_skip == 0:
        #     print(f"Error at iteration {i} is {error}.")
        v = v_new

    if (error > tol) or (max_error > tol*10):
        print(f"{bell.name} failed to converge for k={bell.k}, α = {bell.α},σ ={bell.σ}, i_a={bell.i_a}, and modified θ = {bell.θ + bell.m * (1-bell.θ)} after {i} iterations!")
    elif verbose:
        print(f"Converged in {i} iterations.")
        print(f"Effective k and new c {np.around(bell.grid[bell.index],3),v_greedy[bell.index]}.")
        

    return v_greedy[bell.index],v[bell.index]

 



In [4]:
# Obtain Output for Samples
class agent:
    def __init__(self,
                 k,            # current state k_t
                 θ,            # perceived shock factor θ
                 σ,            # risk averseness
                 α,            # human capital
                 adapt):       # AdapTable 
        self.k,self.θ, self.σ, self.α, self.adapt=k,θ,σ,α,adapt


    
    
# Saltelli
S1_ResultsN=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
S1_ResultsL=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
S1_ResultsH=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
for i in range(len(S_sampledf)):#len(S_sampledf)):
    info=agent(k=S_sampledf.loc[i,"k"], θ=S_sampledf.loc[i,"Theta"], σ=S_sampledf.loc[i,"Sigma"], α=S_sampledf.loc[i,"Alpha"],adapt=AdapTableArray)
    feasible=which_bellman(info)
    if len(feasible)>2:
        S1_ResultsH.loc[i]=[f"S{i}",*feasible[2]]
    if len(feasible)>1:
        S1_ResultsL.loc[i]=[f"S{i}",*feasible[1]]
    S1_ResultsN.loc[i]=[f"S{i}",*feasible[0]]
    

    


In [5]:
print(pd.DataFrame({"N":S1_ResultsN["Value"], "L":S1_ResultsL["Value"], "H":S1_ResultsH["Value"]}))

                 N         L         H
0        -3.394385 -0.507725 -1.612997
1         1.084136  1.101250  0.516426
2        -3.089501 -0.108840 -1.046469
3        -2.646583 -0.474797 -1.508955
4   -169573.406747 -3.360168 -3.120720
..             ...       ...       ...
495       2.685924  3.177098  4.045784
496       2.025169  2.476986  3.187560
497      -0.478427  2.543510  3.334741
498     -62.310492  0.348477  2.680253
499       2.218366  2.718031  3.498606

[500 rows x 3 columns]


In [6]:
#Tol 0.1

def solve_bellman(bell,
                  tol=0.1,
                  min_iter=10,
                  max_iter=1000,
                  verbose=False):
    """
    From: https://python.quantecon.org/optgrowth.html (similar example
    https://macroeconomics.github.io/Dynamic%20Programming.html#.ZC13-exBy3I)
    
    Solve model by iterating with the Bellman operator.

    """


    # Set up loop

    v = bell.u(bell.grid,bell.σ)  # Initial condition
    i = 0
    error = tol + 1
    max_error = tol*10 + 1

    while (i < max_iter and (error > tol or max_error > tol*10)) or (i < min_iter):
        v_greedy, v_new = update_bellman(v, bell)
        error = np.abs(v[bell.index] - v_new)[bell.index]
        max_error = np.max(np.abs(v - v_new))
        i += 1
        # if verbose and i % print_skip == 0:
        #     print(f"Error at iteration {i} is {error}.")
        v = v_new

    if (error > tol) or (max_error > tol*10):
        print(f"{bell.name} failed to converge for k={bell.k}, α = {bell.α},σ ={bell.σ}, i_a={bell.i_a}, and modified θ = {bell.θ + bell.m * (1-bell.θ)} after {i} iterations!")
    elif verbose:
        print(f"Converged in {i} iterations.")
        print(f"Effective k and new c {np.around(bell.grid[bell.index],3),v_greedy[bell.index]}.")
        

    return v_greedy[bell.index],v[bell.index]

 # Obtain Output for Samples
class agent:
    def __init__(self,
                 k,            # current state k_t
                 θ,            # perceived shock factor θ
                 σ,            # risk averseness
                 α,            # human capital
                 adapt):       # AdapTable 
        self.k,self.θ, self.σ, self.α, self.adapt=k,θ,σ,α,adapt


    
    
# Saltelli
STenth_ResultsN=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
STenth_ResultsL=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
STenth_ResultsH=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
for i in range(len(S_sampledf)):#len(S_sampledf)):
    info=agent(k=S_sampledf.loc[i,"k"], θ=S_sampledf.loc[i,"Theta"], σ=S_sampledf.loc[i,"Sigma"], α=S_sampledf.loc[i,"Alpha"],adapt=AdapTableArray)
    feasible=which_bellman(info)
    if len(feasible)>2:
        STenth_ResultsH.loc[i]=[f"S{i}",*feasible[2]]
    if len(feasible)>1:
        STenth_ResultsL.loc[i]=[f"S{i}",*feasible[1]]
    STenth_ResultsN.loc[i]=[f"S{i}",*feasible[0]]


In [7]:
#print(S_ResultsN["Value"])
print(pd.DataFrame({"N":STenth_ResultsN["Value"], "L":STenth_ResultsL["Value"], "H":STenth_ResultsH["Value"]}))

                 N         L          H
0        -3.406300 -5.701979  -8.987846
1         1.081514  0.798566  -0.976655
2        -3.101417 -5.224954  -8.290181
3        -3.127967 -5.226291  -8.156691
4   -169589.913333 -9.786140 -16.005159
..             ...       ...        ...
495       0.292817  0.416264   2.027577
496      -1.010402 -1.092142  -0.257316
497      -1.994235 -1.068638  -0.023402
498     -80.513513 -4.421133  -2.536916
499      -0.817090 -0.845174   0.090783

[500 rows x 3 columns]


In [34]:
#Tol 0.01

def solve_bellman(bell,
                  tol=0.01,
                  min_iter=10,
                  max_iter=2000,
                  verbose=False):
    """
    From: https://python.quantecon.org/optgrowth.html (similar example
    https://macroeconomics.github.io/Dynamic%20Programming.html#.ZC13-exBy3I)
    
    Solve model by iterating with the Bellman operator.

    """


    # Set up loop

    v = bell.u(bell.grid,bell.σ)  # Initial condition
    i = 0
    error = tol + 1
    max_error = tol*10 + 1

    while (i < max_iter and (error > tol or max_error > tol*10)) or (i < min_iter):
        v_greedy, v_new = update_bellman(v, bell)
        error = np.abs(v[bell.index] - v_new)[bell.index]
        max_error = np.max(np.abs(v - v_new))
        i += 1
        # if verbose and i % print_skip == 0:
        #     print(f"Error at iteration {i} is {error}.")
        v = v_new

    if (error > tol) or (max_error > tol*10):
        print(f"{bell.name} failed to converge for k={bell.k}, α = {bell.α},σ ={bell.σ}, i_a={bell.i_a}, and modified θ = {bell.θ + bell.m * (1-bell.θ)} after {i} iterations!")
    elif verbose:
        print(f"Converged in {i} iterations.")
        print(f"Effective k and new c {np.around(bell.grid[bell.index],3),v_greedy[bell.index]}.")
        

    return v_greedy[bell.index],v[bell.index]

# Obtain Output for Samples
class agent:
    def __init__(self,
                 k,            # current state k_t
                 θ,            # perceived shock factor θ
                 σ,            # risk averseness
                 α,            # human capital
                 adapt):       # AdapTable 
        self.k,self.θ, self.σ, self.α, self.adapt=k,θ,σ,α,adapt


    
    
# Saltelli
SHundth_ResultsN=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
SHundth_ResultsL=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
SHundth_ResultsH=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
for i in range(len(S_sampledf)):#len(S_sampledf)):
    info=agent(k=S_sampledf.loc[i,"k"], θ=S_sampledf.loc[i,"Theta"], σ=S_sampledf.loc[i,"Sigma"], α=S_sampledf.loc[i,"Alpha"],adapt=AdapTableArray)
    feasible=which_bellman(info)
    if len(feasible)>2:
        SHundth_ResultsH.loc[i]=[f"S{i}",*feasible[2]]
    if len(feasible)>1:
        SHundth_ResultsL.loc[i]=[f"S{i}",*feasible[1]]
    SHundth_ResultsN.loc[i]=[f"S{i}",*feasible[0]]
 


BellmanNarrowExtended failed to converge for k=6.9207519531249995, α = 1.1164759545940166,σ =-0.2, i_a=0.0, and modified θ = 0.9 after 2000 iterations!
BellmanNarrowExtended failed to converge for k=0.414208984375, α = 1.1025334668114688,σ =-0.2, i_a=0.0, and modified θ = 0.9 after 2000 iterations!
BellmanNarrowExtended failed to converge for k=9.202392578125, α = 1.0544107175248696,σ =-0.1, i_a=0.0, and modified θ = 1.0 after 2000 iterations!
BellmanNarrowExtended failed to converge for k=2.647509765625, α = 1.1649505662161492,σ =-0.2, i_a=0.45, and modified θ = 0.91 after 2000 iterations!
BellmanNarrowExtended failed to converge for k=5.4222167968749995, α = 1.1649505662161492,σ =-0.2, i_a=0.25, and modified θ = 0.95 after 2000 iterations!
BellmanNarrowExtended failed to converge for k=2.647509765625, α = 1.0617942630442245,σ =-0.2, i_a=0.0, and modified θ = 0.9 after 2000 iterations!
BellmanNarrowExtended failed to converge for k=2.647509765625, α = 1.0617942630442245,σ =-0.2, i_a=0

In [35]:
print(pd.DataFrame({"N":SHundth_ResultsN["Value"], "L":SHundth_ResultsL["Value"], "H":SHundth_ResultsH["Value"]}))
SHundth_ResultsN.to_csv("ResultsNone.csv")
SHundth_ResultsL.to_csv("ResultsLow.csv")
SHundth_ResultsH.to_csv("ResultsHigh.csv")

HundthIndex = pd.DataFrame({"N":SHundth_ResultsN["Value"], "L":SHundth_ResultsL["Value"], "H":SHundth_ResultsH["Value"]}).apply('idxmin', axis=1)
HundthCon = pd.DataFrame({"N":SHundth_ResultsN["Consumption"], "L":SHundth_ResultsL["Consumption"], "H":SHundth_ResultsH["Consumption"]})

HundthResult = [HundthCon.loc[i, HundthIndex.loc[i]] for i in range(len(HundthIndex))]
HundthFinal= pd.DataFrame({"Equation":HundthIndex,"Consumption": HundthResult})

HundthFinal.to_csv("ResultsFinal.csv")



                  N          L          H
0     -1.170888e+02 -10.736273 -16.972574
1     -6.396648e+01  -7.760943  -6.573503
2     -1.105234e+02  -8.622154 -12.347946
3     -4.019631e+01 -10.675534 -14.711720
4     -3.441186e+01 -10.116293 -15.881917
...             ...        ...        ...
10235 -2.778606e+01 -12.283369 -11.606918
10236 -2.782736e+01  -9.810337  -5.716766
10237 -4.013776e+06 -10.754461  -8.779354
10238 -2.572163e+01  -6.515430  -3.631795
10239 -2.768343e+01  -9.644542  -5.481222

[10240 rows x 3 columns]


In [13]:
#Tol 0.001

def solve_bellman(bell,
                  tol=0.001,
                  min_iter=10,
                  max_iter=3000,
                  verbose=False):
    """
    From: https://python.quantecon.org/optgrowth.html (similar example
    https://macroeconomics.github.io/Dynamic%20Programming.html#.ZC13-exBy3I)
    
    Solve model by iterating with the Bellman operator.

    """


    # Set up loop

    v = bell.u(bell.grid,bell.σ)  # Initial condition
    i = 0
    error = tol + 1
    max_error = tol*10 + 1

    while (i < max_iter and (error > tol or max_error > tol*10)) or (i < min_iter):
        v_greedy, v_new = update_bellman(v, bell)
        error = np.abs(v[bell.index] - v_new)[bell.index]
        max_error = np.max(np.abs(v - v_new))
        i += 1
        # if verbose and i % print_skip == 0:
        #     print(f"Error at iteration {i} is {error}.")
        v = v_new

    if (error > tol) or (max_error > tol*10):
        print(f"{bell.name} failed to converge for k={bell.k}, α = {bell.α},σ ={bell.σ}, i_a={bell.i_a}, and modified θ = {bell.θ + bell.m * (1-bell.θ)} after {i} iterations!")
    elif verbose:
        print(f"Converged in {i} iterations.")
        print(f"Effective k and new c {np.around(bell.grid[bell.index],3),v_greedy[bell.index]}.")
        

    return v_greedy[bell.index],v[bell.index]

# Obtain Output for Samples
class agent:
    def __init__(self,
                 k,            # current state k_t
                 θ,            # perceived shock factor θ
                 σ,            # risk averseness
                 α,            # human capital
                 adapt):       # AdapTable 
        self.k,self.θ, self.σ, self.α, self.adapt=k,θ,σ,α,adapt


    
    
# Saltelli
SThouth_ResultsN=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
SThouth_ResultsL=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
SThouth_ResultsH=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
for i in range(len(S_sampledf)):#len(S_sampledf)):
    info=agent(k=S_sampledf.loc[i,"k"], θ=S_sampledf.loc[i,"Theta"], σ=S_sampledf.loc[i,"Sigma"], α=S_sampledf.loc[i,"Alpha"],adapt=AdapTableArray)
    feasible=which_bellman(info)
    if len(feasible)>2:
        SThouth_ResultsH.loc[i]=[f"S{i}",*feasible[2]]
    if len(feasible)>1:
        SThouth_ResultsL.loc[i]=[f"S{i}",*feasible[1]]
    SThouth_ResultsN.loc[i]=[f"S{i}",*feasible[0]]
 


BellmanNarrowExtended failed to converge for k=2.4976562500000004, α = 1.1281077012279517,σ =-0.2, i_a=0.0, and modified θ = 0.9 after 3000 iterations!
BellmanNarrowExtended failed to converge for k=2.4976562500000004, α = 1.1281077012279517,σ =-0.2, i_a=0.25, and modified θ = 0.95 after 3000 iterations!
BellmanNarrowExtended failed to converge for k=2.4976562500000004, α = 1.0421885303013698,σ =-0.2, i_a=0.0, and modified θ = 0.8 after 3000 iterations!
BellmanNarrowExtended failed to converge for k=2.4976562500000004, α = 1.0421885303013698,σ =-0.2, i_a=0.45, and modified θ = 0.98 after 3000 iterations!
BellmanNarrowExtended failed to converge for k=4.19921875, α = 1.0421885303013698,σ =-0.2, i_a=0.0, and modified θ = 0.9 after 3000 iterations!
BellmanNarrowExtended failed to converge for k=4.19921875, α = 1.0421885303013698,σ =-0.2, i_a=0.0, and modified θ = 0.8 after 3000 iterations!


In [14]:
print(pd.DataFrame({"N":SThouth_ResultsN["Value"], "L":SThouth_ResultsL["Value"], "H":SThouth_ResultsH["Value"]}))

                 N          L          H
0        -3.408028  -7.552071 -10.840466
1         1.081112  -1.289895  -2.818717
2        -3.103145  -7.074379 -10.142249
3        -3.197720  -7.069757 -10.026376
4   -169591.717026 -11.643219 -19.646231
..             ...        ...        ...
495      -0.492900  -1.473721   0.166545
496      -2.007701  -2.915935  -2.061192
497      -2.227449  -2.975682  -1.886542
498     -82.332318  -6.233959  -4.393053
499      -1.814389  -2.668771  -1.712757

[500 rows x 3 columns]


In [15]:
#Tol 0.1 just Plus 10 on the grid


class BellmanEquation:
     #Adapted from: https://python.quantecon.org/optgrowth.html
    def __init__(self,
                 u,            # utility function
                 f,            # production function
                 k,            # current state k_t
                 θ,            # given shock factor θ
                 σ,            # risk averseness
                 α,            # human capital
                 i_a,          # adaptation investment
                 m,            # protection multiplier
                 β=β,          # discount factor
                 𝛿=𝛿,          # depreciation factor 
                 name="BellmanNarrowExtended"):

        self.u, self.f, self.k, self.β, self.θ, self.𝛿, self.σ, self.α, self.i_a, self.m, self.name = u, f, k, β, θ, 𝛿, σ, α, i_a, m, name

        # Set up grid
        
        startgrid=np.array([1.0e-7,1,2,3,4,5,6,7,8,9,10,k+10])

        ind=np.searchsorted(startgrid, k)
        self.grid=np.concatenate((startgrid[:ind],np.array([k*0.99999, k]),
                                 startgrid[ind:]))

        self.grid=self.grid[self.grid>i_a]

        # Identify target state k
        self.index = np.searchsorted(self.grid, k)-1
    
    def value(self, c, y, v_array):
        """
        Right hand side of the Bellman equation.
        """

        u, f, β, θ, 𝛿, σ, α, i_a, m = self.u, self.f, self.β, self.θ, self.𝛿, self.σ, self.α, self.i_a, self.m

        v = interp1d(self.grid, v_array, bounds_error=False, 
                     fill_value="extrapolate")
        
        return u(c,σ) + β * v((θ + m * (1-θ)) * (f(y,α) - c - i_a + (1 - 𝛿) * y))



def update_bellman(v, bell):
    """
    From: https://python.quantecon.org/optgrowth.html (similar example
    https://macroeconomics.github.io/Dynamic%20Programming.html#.ZC13-exBy3I)
    
    The Bellman operator.  Updates the guess of the value function
    and also computes a v-greedy policy.

      * bell is an instance of Bellman equation
      * v is an array representing a guess of the value function

    """
    v_new = np.empty_like(v)
    v_greedy = np.empty_like(v)
    
    for i in range(len(bell.grid)):
        y = bell.grid[i]
        # Maximize RHS of Bellman equation at state y
        
        c_star, v_max = maximize(bell.value, min([1e-8,y*0.00001]), 
                                 y-bell.i_a, (y, v))
        #VMG HELP! can anyone check that (1) subtracting i_a and 
        # (2) omitting any grid values less than i_a 
        # will not be problematic? The only thing I can come up with
        # is if i_a is greater than k*0.99999
        # which_bellman() now accounts for that case. Whole thing 
        # could use refinement.
      
        v_new[i] = v_max
        v_greedy[i] = c_star

    return v_greedy, v_new

def which_bellman(agentinfo):
    """
    Solves bellman for each affordable adaptation option.
    """
    feasible=[]


    for option in agentinfo.adapt:
        if option[1]>=(income_function(agentinfo.k,agentinfo.α)+(1-𝛿)*agentinfo.k)*0.99998:
            # ensures that the gridpoint
            # just below income, income*0.99999, is included
            pass
        else:
            #print(f'working theta = {agentinfo.θ + option[0] *\
            #  (1-agentinfo.θ)}, i_a= {option[1]}, k= {agentinfo.k}')
            c,v=solve_bellman(BellmanEquation(u=utility, 
                              f=income_function, k=agentinfo.k, 
                              θ=agentinfo.θ, σ=agentinfo.σ, 
                              α=agentinfo.α, i_a=option[1],m=option[0]))
            feasible.append([v,c,option[1],option[0]])

    best=min(feasible)

    
    return feasible



def solve_bellman(bell,
                  tol=0.1,
                  min_iter=10,
                  max_iter=1000,
                  verbose=False):
    """
    From: https://python.quantecon.org/optgrowth.html (similar example
    https://macroeconomics.github.io/Dynamic%20Programming.html#.ZC13-exBy3I)
    
    Solve model by iterating with the Bellman operator.

    """


    # Set up loop

    v = bell.u(bell.grid,bell.σ)  # Initial condition
    i = 0
    error = tol + 1
    max_error = tol*10 + 1

    while (i < max_iter and (error > tol or max_error > tol*10)) or (i < min_iter):
        v_greedy, v_new = update_bellman(v, bell)
        error = np.abs(v[bell.index] - v_new)[bell.index]
        max_error = np.max(np.abs(v - v_new))
        i += 1
        # if verbose and i % print_skip == 0:
        #     print(f"Error at iteration {i} is {error}.")
        v = v_new

    if (error > tol) or (max_error > tol*10):
        print(f"{bell.name} failed to converge for k={bell.k}, α = {bell.α},σ ={bell.σ}, i_a={bell.i_a}, and modified θ = {bell.θ + bell.m * (1-bell.θ)} after {i} iterations!")
    elif verbose:
        print(f"Converged in {i} iterations.")
        print(f"Effective k and new c {np.around(bell.grid[bell.index],3),v_greedy[bell.index]}.")
        

    return v_greedy[bell.index],v[bell.index]




# Obtain Output for Samples
class agent:
    def __init__(self,
                 k,            # current state k_t
                 θ,            # perceived shock factor θ
                 σ,            # risk averseness
                 α,            # human capital
                 adapt):       # AdapTable 
        self.k,self.θ, self.σ, self.α, self.adapt=k,θ,σ,α,adapt


    
    
# Saltelli
SGrid10_ResultsN=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
SGrid10_ResultsL=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
SGrid10_ResultsH=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
for i in range(len(S_sampledf)):#len(S_sampledf)):
    info=agent(k=S_sampledf.loc[i,"k"], θ=S_sampledf.loc[i,"Theta"], σ=S_sampledf.loc[i,"Sigma"], α=S_sampledf.loc[i,"Alpha"],adapt=AdapTableArray)
    feasible=which_bellman(info)
    if len(feasible)>2:
        SGrid10_ResultsH.loc[i]=[f"S{i}",*feasible[2]]
    if len(feasible)>1:
        SGrid10_ResultsL.loc[i]=[f"S{i}",*feasible[1]]
    SGrid10_ResultsN.loc[i]=[f"S{i}",*feasible[0]]
 

 



In [16]:
print(pd.DataFrame({"N":SGrid10_ResultsN["Value"], "L":SGrid10_ResultsL["Value"], "H":SGrid10_ResultsH["Value"]}))

                 N         L          H
0        -3.406300 -5.701979  -8.987846
1         1.081514  0.798566  -0.976655
2        -3.101417 -5.224954  -8.290181
3        -3.127967 -5.226291  -8.156691
4   -169589.913333 -9.786140 -16.005159
..             ...       ...        ...
495       0.292817  0.416264   2.027577
496      -1.010402 -1.092142  -0.257316
497      -1.994235 -1.068638  -0.023402
498     -80.513513 -4.421133  -2.536916
499      -0.817090 -0.845174   0.090783

[500 rows x 3 columns]


In [29]:
HundthIndex = pd.DataFrame({"N":SHundth_ResultsN["Value"], "L":SHundth_ResultsL["Value"], "H":SHundth_ResultsH["Value"]}).apply('idxmin', axis=1)
TenthIndex = pd.DataFrame({"N":STenth_ResultsN["Value"], "L":STenth_ResultsL["Value"], "H":STenth_ResultsH["Value"]}).apply('idxmin', axis=1)
ThouthIndex = pd.DataFrame({"N":SThouth_ResultsN["Value"], "L":SThouth_ResultsL["Value"], "H":SThouth_ResultsH["Value"]}).apply('idxmin', axis=1)
HundthCon = pd.DataFrame({"N":SHundth_ResultsN["Consumption"], "L":SHundth_ResultsL["Consumption"], "H":SHundth_ResultsH["Consumption"]})
TenthCon = pd.DataFrame({"N":STenth_ResultsN["Consumption"], "L":STenth_ResultsL["Consumption"], "H":STenth_ResultsH["Consumption"]})
ThouthCon = pd.DataFrame({"N":SThouth_ResultsN["Consumption"], "L":SThouth_ResultsL["Consumption"], "H":SThouth_ResultsH["Consumption"]})

HundthResult = [HundthCon.loc[i, HundthIndex.loc[i]] for i in range(len(HundthIndex))]
ThouthResult = [ThouthCon.loc[i, ThouthIndex.loc[i]] for i in range(len(ThouthIndex))]
print(pd.DataFrame({"Hund":HundthResult,"Thou":ThouthResult}))



         Hund      Thou
0    0.721988  0.721987
1    0.906963  0.906383
2    0.788278  0.788278
3    0.741651  0.741651
4    0.006341  0.006341
..        ...       ...
495  2.217495  2.217494
496  2.062920  2.062910
497  2.028719  2.028695
498  1.376168  1.376160
499  2.266383  2.266372

[500 rows x 2 columns]




# LHS
LH_ResultsN=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
LH_ResultsL=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
LH_ResultsH=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
for i in range(len(LH_sampledf)):
    info=agent(k=LH_sampledf.loc[i,"k"], θ=LH_sampledf.loc[i,"Theta"], σ=LH_sampledf.loc[i,"Sigma"], α=LH_sampledf.loc[i,"Alpha"],adapt=AdapTableArray)
    feasible=which_bellman(info)
    if len(feasible)>2:
        LH_ResultsH.loc[i]=[f"S{i}",*feasible[2]]
    if len(feasible)>1:
        LH_ResultsL.loc[i]=[f"S{i}",*feasible[1]]
    LH_ResultsN.loc[i]=[f"S{i}",*feasible[0]]
    




# Random

R_ResultsN=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
R_ResultsL=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
R_ResultsH=pd.DataFrame({'Agent':[],'Value':[],'Consumption':[], 'i_a':[], 'm':[]})
for i in range(len(R_sampledf)):
    info=agent(k=R_sampledf.loc[i,"k"], θ=R_sampledf.loc[i,"Theta"], σ=R_sampledf.loc[i,"Sigma"], α=R_sampledf.loc[i,"Alpha"], adapt=AdapTableArray)
    feasible=which_bellman(info)
    if len(feasible)>2:
        R_ResultsH.loc[i]=[f"S{i}",*feasible[2]]
    if len(feasible)>1:
        R_ResultsL.loc[i]=[f"S{i}",*feasible[1]]
    R_ResultsN.loc[i]=[f"S{i}",*feasible[0]]

In [ ]:
S_ResultsN.to_csv(f"SaltelliResultsNone.csv")
S_ResultsL.to_csv(f"SaltelliResultsLow.csv")
S_ResultsH.to_csv(f"SaltelliResultsHigh.csv")

# LH_ResultsN.to_csv(f"LatinResultsNone.csv")
# LH_ResultsL.to_csv(f"LatinResultsLow.csv")
# LH_ResultsH.to_csv(f"LatinResultsHigh.csv")

# R_ResultsN.to_csv(f"RandomResultsNone.csv")
# R_ResultsL.to_csv(f"RandomResultsLow.csv")
# R_ResultsH.to_csv(f"RandomResultsHigh.csv")

# Create Interpolated Surface (Linear)
print(len(S_sampledf["k"]),len(S_ResultsN["Value"]))
# Saltelli
S_N_Interp_C=griddata((S_sampledf["k"],S_sampledf["Theta"], S_sampledf["Sigma"],S_sampledf["Alpha"]),S_ResultsN["Consumption"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')
S_N_Interp_V=griddata((S_sampledf["k"],S_sampledf["Theta"], S_sampledf["Sigma"],S_sampledf["Alpha"]),S_ResultsN["Value"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')

S_L_Interp_C=griddata((S_sampledf["k"],S_sampledf["Theta"], S_sampledf["Sigma"],S_sampledf["Alpha"]),S_ResultsL["Consumption"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')
S_L_Interp_V=griddata((S_sampledf["k"],S_sampledf["Theta"], S_sampledf["Sigma"],S_sampledf["Alpha"]),S_ResultsL["Value"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')

S_H_Interp_C=griddata((S_sampledf["k"],S_sampledf["Theta"], S_sampledf["Sigma"],S_sampledf["Alpha"]),S_ResultsH["Consumption"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')
S_H_Interp_V=griddata((S_sampledf["k"],S_sampledf["Theta"], S_sampledf["Sigma"],S_sampledf["Alpha"]),S_ResultsH["Value"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')

# LHS
LH_N_Interp=griddata((LH_sampledf["k"],LH_sampledf["Theta"], LH_sampledf["Sigma"],LH_sampledf["Alpha"]),LH_ResultsN["Consumption"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')
LH_N_Interp=griddata((LH_sampledf["k"],LH_sampledf["Theta"], LH_sampledf["Sigma"],LH_sampledf["Alpha"]),LH_ResultsN["Value"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')

LH_L_Interp=griddata((LH_sampledf["k"],LH_sampledf["Theta"], LH_sampledf["Sigma"],LH_sampledf["Alpha"]),LH_ResultsL["Consumption"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')
LH_L_Interp=griddata((LH_sampledf["k"],LH_sampledf["Theta"], LH_sampledf["Sigma"],LH_sampledf["Alpha"]),LH_ResultsL["Value"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')

LH_H_Interp=griddata((LH_sampledf["k"],LH_sampledf["Theta"], LH_sampledf["Sigma"],LH_sampledf["Alpha"]),LH_ResultsH["Consumption"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')
LH_H_Interp=griddata((LH_sampledf["k"],LH_sampledf["Theta"], LH_sampledf["Sigma"],LH_sampledf["Alpha"]),LH_ResultsH["Value"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')


# Both, bc why not?
SLH_N_Interp=griddata((pd.concat([S_sampledf["k"],LH_sampledf["k"]]),pd.concat([S_sampledf["Theta"],LH_sampledf["Theta"]]), pd.concat([S_sampledf["Sigma"],LH_sampledf["Sigma"]]),pd.concat([S_sampledf["Alpha"],LH_sampledf["Alpha"]])),pd.concat([S_ResultsN["Consumption"],LH_ResultsN["Consumption"]]),(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')
SLH_N_Interp=griddata((pd.concat([S_sampledf["k"],LH_sampledf["k"]]),pd.concat([S_sampledf["Theta"],LH_sampledf["Theta"]]), pd.concat([S_sampledf["Sigma"],LH_sampledf["Sigma"]]),pd.concat([S_sampledf["Alpha"],LH_sampledf["Alpha"]])),pd.concat([S_ResultsN["Value"],LH_ResultsN["Value"]]),(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')

SLH_L_Interp=griddata((pd.concat([S_sampledf["k"],LH_sampledf["k"]]),pd.concat([S_sampledf["Theta"],LH_sampledf["Theta"]]), pd.concat([S_sampledf["Sigma"],LH_sampledf["Sigma"]]),pd.concat([S_sampledf["Alpha"],LH_sampledf["Alpha"]])),pd.concat([S_ResultsN["Consumption"],LH_ResultsN["Consumption"]]),(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')
SLH_L_Interp=griddata((pd.concat([S_sampledf["k"],LH_sampledf["k"]]),pd.concat([S_sampledf["Theta"],LH_sampledf["Theta"]]), pd.concat([S_sampledf["Sigma"],LH_sampledf["Sigma"]]),pd.concat([S_sampledf["Alpha"],LH_sampledf["Alpha"]])),pd.concat([S_ResultsN["Value"],LH_ResultsN["Value"]]),(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')

SLH_H_Interp=griddata((pd.concat([S_sampledf["k"],LH_sampledf["k"]]),pd.concat([S_sampledf["Theta"],LH_sampledf["Theta"]]), pd.concat([S_sampledf["Sigma"],LH_sampledf["Sigma"]]),pd.concat([S_sampledf["Alpha"],LH_sampledf["Alpha"]])),pd.concat([S_ResultsN["Consumption"],LH_ResultsN["Consumption"]]),(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')
SLH_H_Interp=griddata((pd.concat([S_sampledf["k"],LH_sampledf["k"]]),pd.concat([S_sampledf["Theta"],LH_sampledf["Theta"]]), pd.concat([S_sampledf["Sigma"],LH_sampledf["Sigma"]]),pd.concat([S_sampledf["Alpha"],LH_sampledf["Alpha"]])),pd.concat([S_ResultsN["Value"],LH_ResultsN["Value"]]),(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='linear')



In [ ]:
# Create Interpolated Surface (Nearest)
print(len(S_sampledf["k"]),len(S_ResultsN["Value"]))
# Saltelli
S_N_Interp_C=griddata((S_sampledf["k"],S_sampledf["Theta"], S_sampledf["Sigma"],S_sampledf["Alpha"]),S_ResultsN["Consumption"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')
S_N_Interp_V=griddata((S_sampledf["k"],S_sampledf["Theta"], S_sampledf["Sigma"],S_sampledf["Alpha"]),S_ResultsN["Value"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')

S_L_Interp_C=griddata((S_sampledf["k"],S_sampledf["Theta"], S_sampledf["Sigma"],S_sampledf["Alpha"]),S_ResultsL["Consumption"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')
S_L_Interp_V=griddata((S_sampledf["k"],S_sampledf["Theta"], S_sampledf["Sigma"],S_sampledf["Alpha"]),S_ResultsL["Value"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')

S_H_Interp_C=griddata((S_sampledf["k"],S_sampledf["Theta"], S_sampledf["Sigma"],S_sampledf["Alpha"]),S_ResultsH["Consumption"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')
S_H_Interp_V=griddata((S_sampledf["k"],S_sampledf["Theta"], S_sampledf["Sigma"],S_sampledf["Alpha"]),S_ResultsH["Value"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')

# LHS
LH_N_Interp_C=griddata((LH_sampledf["k"],LH_sampledf["Theta"], LH_sampledf["Sigma"],LH_sampledf["Alpha"]),LH_ResultsN["Consumption"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')
LH_N_Interp_V=griddata((LH_sampledf["k"],LH_sampledf["Theta"], LH_sampledf["Sigma"],LH_sampledf["Alpha"]),LH_ResultsN["Value"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')

LH_L_Interp_C=griddata((LH_sampledf["k"],LH_sampledf["Theta"], LH_sampledf["Sigma"],LH_sampledf["Alpha"]),LH_ResultsL["Consumption"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')
LH_L_Interp_V=griddata((LH_sampledf["k"],LH_sampledf["Theta"], LH_sampledf["Sigma"],LH_sampledf["Alpha"]),LH_ResultsL["Value"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')

LH_H_Interp_C=griddata((LH_sampledf["k"],LH_sampledf["Theta"], LH_sampledf["Sigma"],LH_sampledf["Alpha"]),LH_ResultsH["Consumption"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')
LH_H_Interp_V=griddata((LH_sampledf["k"],LH_sampledf["Theta"], LH_sampledf["Sigma"],LH_sampledf["Alpha"]),LH_ResultsH["Value"],(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')


# Both, bc why not?
SLH_N_Interp_C=griddata((pd.concat([S_sampledf["k"],LH_sampledf["k"]]),pd.concat([S_sampledf["Theta"],LH_sampledf["Theta"]]), pd.concat([S_sampledf["Sigma"],LH_sampledf["Sigma"]]),pd.concat([S_sampledf["Alpha"],LH_sampledf["Alpha"]])),pd.concat([S_ResultsN["Consumption"],LH_ResultsN["Consumption"]]),(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')
SLH_N_Interp_V=griddata((pd.concat([S_sampledf["k"],LH_sampledf["k"]]),pd.concat([S_sampledf["Theta"],LH_sampledf["Theta"]]), pd.concat([S_sampledf["Sigma"],LH_sampledf["Sigma"]]),pd.concat([S_sampledf["Alpha"],LH_sampledf["Alpha"]])),pd.concat([S_ResultsN["Value"],LH_ResultsN["Value"]]),(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')

SLH_L_Interp_C=griddata((pd.concat([S_sampledf["k"],LH_sampledf["k"]]),pd.concat([S_sampledf["Theta"],LH_sampledf["Theta"]]), pd.concat([S_sampledf["Sigma"],LH_sampledf["Sigma"]]),pd.concat([S_sampledf["Alpha"],LH_sampledf["Alpha"]])),pd.concat([S_ResultsN["Consumption"],LH_ResultsN["Consumption"]]),(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')
SLH_L_Interp_V=griddata((pd.concat([S_sampledf["k"],LH_sampledf["k"]]),pd.concat([S_sampledf["Theta"],LH_sampledf["Theta"]]), pd.concat([S_sampledf["Sigma"],LH_sampledf["Sigma"]]),pd.concat([S_sampledf["Alpha"],LH_sampledf["Alpha"]])),pd.concat([S_ResultsN["Value"],LH_ResultsN["Value"]]),(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')

SLH_H_Interp_C=griddata((pd.concat([S_sampledf["k"],LH_sampledf["k"]]),pd.concat([S_sampledf["Theta"],LH_sampledf["Theta"]]), pd.concat([S_sampledf["Sigma"],LH_sampledf["Sigma"]]),pd.concat([S_sampledf["Alpha"],LH_sampledf["Alpha"]])),pd.concat([S_ResultsN["Consumption"],LH_ResultsN["Consumption"]]),(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')
SLH_H_Interp_V=griddata((pd.concat([S_sampledf["k"],LH_sampledf["k"]]),pd.concat([S_sampledf["Theta"],LH_sampledf["Theta"]]), pd.concat([S_sampledf["Sigma"],LH_sampledf["Sigma"]]),pd.concat([S_sampledf["Alpha"],LH_sampledf["Alpha"]])),pd.concat([S_ResultsN["Value"],LH_ResultsN["Value"]]),(R_sampledf["k"],R_sampledf["Theta"], R_sampledf["Sigma"],R_sampledf["Alpha"]), method='nearest')





In [ ]:
import torch
S_Interp_V=torch.stack([torch.tensor(S_N_Interp_V),torch.tensor(S_L_Interp_V),torch.tensor(S_H_Interp_V)], dim=1)
S_Interp_C=torch.stack([torch.tensor(S_N_Interp_C),torch.tensor(S_L_Interp_C),torch.tensor(S_H_Interp_C)], dim=1)
S_Interp_max=torch.max(S_Interp_V,dim=1)

S_consumption=S_Interp_C*torch.nn.functional.one_hot(S_Interp_max[1])
S_consumption=S_consumption.sum(dim=1)

S_adaptation=S_Interp_max[0]

print(S_Interp_V)



In [ ]:
LH_Interp_V=torch.stack([torch.tensor(LH_N_Interp_V),torch.tensor(LH_L_Interp_V),torch.tensor(LH_H_Interp_V)], dim=1)
LH_Interp_C=torch.stack([torch.tensor(LH_N_Interp_C),torch.tensor(LH_L_Interp_C),torch.tensor(LH_H_Interp_C)], dim=1)
LH_Interp_max=torch.max(LH_Interp_V,dim=1)

LH_consumption=LH_Interp_C*torch.nn.functional.one_hot(LH_Interp_max[1])
LH_consumption=LH_consumption.sum(dim=1)

LH_adaptation=LH_Interp_max[0]

In [ ]:
SLH_Interp_V=torch.stack([torch.tensor(SLH_N_Interp_V),torch.tensor(SLH_L_Interp_V),torch.tensor(SLH_H_Interp_V)], dim=1)
SLH_Interp_C=torch.stack([torch.tensor(SLH_N_Interp_C),torch.tensor(SLH_L_Interp_C),torch.tensor(SLH_H_Interp_C)], dim=1)
SLH_Interp_max=torch.max(SLH_Interp_V,dim=1)

SLH_consumption=SLH_Interp_C*torch.nn.functional.one_hot(SLH_Interp_max[1])
SLH_consumption=SLH_consumption.sum(dim=1)

SLH_adaptation=SLH_Interp_max[0]

In [ ]:
# Test Excluded Sample 
import matplotlib.pyplot as plt

plt.scatter(x=R_sampledf.loc[0:99,'k'], y=S_N_Interp, c='blue', s=5, alpha=0.3)
plt.scatter(x=R_sampledf.loc[0:99,'k'], y=R_ResultsN['Consumption'], c='black', s=1)
plt.ylabel("Consumption")
plt.xlabel("k")

plt.show()


plt.scatter(x=R_sampledf.loc[0:99,'k'], y=S_N_Interp, c=range(100), s=7, alpha=0.3)
plt.scatter(x=R_sampledf.loc[0:99,'k'], y=R_ResultsN['Consumption'], c=range(100), s=1)
plt.xlabel("k")
plt.ylabel("Consumption")

plt.show()



MAE=abs(S_N_Interp_C-R_ResultsN['Consumption'])
plt.boxplot(MAE)
plt.ylabel('MAE')
plt.show()

#plt.scatter(x=R_sampledf['k'], y=S_N_Interp, c=R_sampledf['AgentID'], s=5, alpha=0.3)
#plt.scatter(x=R_sampledf['k'], y=R_ResultsN['Consumption'], c=R_sampledf['AgentID'], s=1)

#plt.show()






In [ ]:
print(type(pd.read_csv("SaltelliResultsNone.csv")))
S_None=torch.from_numpy(np.genfromtxt("SaltelliResultsNone.csv", delimiter=","))
S_Low=torch.from_numpy(np.genfromtxt("SaltelliResultsLow.csv", delimiter=","))
S_High=torch.from_numpy(np.genfromtxt("SaltelliResultsHigh.csv", delimiter=","))

In [ ]:
from scipy.optimize import curve_fit

x = S_sampledf['k']
y = S_ResultsN["Consumption"]

def f(x,A):
    return A*np.log(x)+1
    

popt, pcov = curve_fit(f, x, y, method="trf")
y_fit = f(x, *popt)
print(popt)

fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(x, y, 'o')
ax.plot(x, y_fit, '.')
plt.show()
